# 03 — LoRA Model Setup

Demonstrates the LoRA (Low-Rank Adaptation) configuration used for fine-tuning
**Qwen2.5-3B-Instruct**.

### Configuration Choices

| Parameter | Value | Rationale |
|---|---|---|
| `lora_rank` | 32 | Good capacity while staying memory-friendly |
| `lora_alpha` | 64 | 2 × rank — standard heuristic for stable learning |
| `lora_dropout` | 0.1 | Light regularisation to prevent overfitting |
| `target_modules` | q/k/v/o_proj + gate/up/down_proj | All key linear layers in attention + MLP |
| `use_4bit` | True | QLoRA — reduces ~6 GB (fp16) to ~2 GB (4-bit) |

In [ ]:
# Setup
import subprocess, sys
try:
    import peft
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
        "torch", "transformers>=4.40", "accelerate",
        "peft>=0.10", "bitsandbytes>=0.43"])

import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [ ]:
from src.lora_setup import LoraSetup, build_trainable_lora_model

cfg = LoraSetup(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    lora_rank=32,
    lora_alpha=64,
    lora_dropout=0.05,
    use_4bit=True,
)

print("LoRA configuration:")
print(f"  Model:          {cfg.model_name}")
print(f"  Rank:           {cfg.lora_rank}")
print(f"  Alpha:          {cfg.lora_alpha}")
print(f"  Dropout:        {cfg.lora_dropout}")
print(f"  Target modules: {cfg.target_modules_requested}")
print(f"  4-bit quant:    {cfg.use_4bit}")

In [ ]:
model, tokenizer, lora_config = build_trainable_lora_model(cfg)

print(f"\nModel device: {model.device}")
print(f"\nLoRA config object:\n{lora_config}")

### Why these choices?

- **Rank 32**: Good middle ground — rank 8/16 may lack capacity for learning
  the letter-counting reasoning pattern; rank 64/128 uses more memory with
  diminishing returns for this relatively simple task.
- **All 7 target modules**: Targeting both attention *and* MLP layers gives
  the adapter maximum expressiveness to learn the counting behaviour.
- **4-bit quantization**: Reduces the frozen base model from ~6 GB to ~2 GB,
  making it feasible to train on GPUs with 8–16 GB VRAM.